# 뉴스 기사 크롤링

In [ ]:
import requests
import datetime
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm.notebook import tqdm #pip install tqdm
link_list = [];titles = [];dates = [];weeks = [];times=[]
weekDays = ("월","화","수","목","금","토","일")


#종목코드
code = '005380'
#크롤링할 페이지
startpage = 1
lastpage = 612

#크롤링
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
print('페이지로딩')
for page in tqdm(range(startpage,lastpage+1)):
    res = requests.get(f'https://finance.naver.com/item/news_news.nhn?code={code}&page={page}', headers = headers)
    soup = bs(res.content, 'lxml')
    href_list = soup.select('td.title > a')
    for i in href_list:
        link_list.append("https://finance.naver.com"+i['href'])      
print("크롤링시작")
for j in tqdm(link_list):
    link_res = requests.get(j, headers = headers)
    soup = bs(link_res.content, 'lxml')
    try:
        news_title = soup.select_one('th > strong').text  
        date , time = soup.select_one('th > span > span').text.split()
        year , month , day = date.split('.')
        get_weekday = datetime.date(int(year),int(month),int(day))
        hour, minute = time.split(':')   
        time = (60 * int(hour) + int(minute))//15  
        weeks.append(weekDays[get_weekday.weekday()]) 
        date = date.replace('.','')
        titles.append(news_title)
        dates.append(int(date))       
        times.append(time)
    except:
        continue
   
    
    
#dataframe  
dic = {'날짜':dates,'요일':weeks,'시간':times, '제목':titles}
crawling_data = pd.DataFrame(dic)
#저장
crawling_data.to_csv('D:/Users/A1-102-u12/210222/20년3월 현대차.csv',encoding="",index=False)

페이지로딩



크롤링시작


In [53]:
#dataframe  
dic = {'날짜':dates,'요일':weeks,'시간':times, '제목':titles}
crawling_data = pd.DataFrame(dic)
#저장
crawling_data.to_csv('D:/Users/A1-102-u12/210222/20년3월 HMM.csv',encoding="",index=False)

In [77]:
#지금까지했던거 합쳐서 데이터셋 만들기
import pandas as pd
df1 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/넷마블_완료.csv", encoding="euc-kr")
df2 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/GS건설_완료.csv", encoding="euc-kr")
df3 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/더존비즈온_완료.csv", encoding="euc-kr")
df4 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/삼성중공업_완료.csv", encoding="euc-kr")
df5 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/에넥스_완료.csv", encoding="euc-kr")
df6 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/위니아딤채_완료.csv", encoding="euc-kr")
df7 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/한국전력_완료.csv", encoding="euc-kr")
df8 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/코오롱_완료.csv", encoding="euc-kr")
df9 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/한국항공우주_완료.csv", encoding="euc-kr")
df10 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/한화생명_완료.csv", encoding="euc-kr")
df11 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/한화솔루션_완료.csv", encoding="euc-kr")
df12 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/현대건설_완료.csv", encoding="euc-kr")
df13 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/신라젠_완료.csv", encoding="euc-kr")
df14 = pd.read_csv("D:/Users/A1-102-u12/210219/2월19일완료/SK하이닉스_완료.csv", encoding="euc-kr")
df15 = pd.read_csv("D:/Users/A1-102-u12/210218_dataset.csv", encoding='euc-kr')
dataset = pd.concat([df1, df2, df3, df4, df5, df6, df7,df8, df9,df10,df11,df12,df13,df14,df15])
dataset= dataset.reset_index(drop=True)
dataset.to_csv('210219_dataset.csv',encoding="",index=False)

In [35]:
import pandas as pd
import numpy as np
#주식데이터 encoding="euc-kr"
data = pd.read_csv("C:/Users/young/210222주식/휴마시스.csv",encoding="euc-kr")
#날짜 수정
dateList=[]; timeList=[]; 
for i in range(len(data)):
    try:
        dateList.append(int(data['날짜'][i].replace("-","")))
    except:
        dateList.append(int(data['날짜'][i].replace("/","")))
data['Date'] = dateList

for i in range(len(data)):
    hour, minute, sec = data['시간'][i].split(':')
    timeList.append((60*int(hour)+int(minute))//15)
data['Time'] = timeList


#15분사이 변하는 값
data['Close-Open'] = data['종가']-data['시가']


#15분사이 상승 = 1
#하락 = -1
#변화없음 = 0
data['C-O'] = 0
data.loc[data['Close-Open'] > 0,'C-O'] = 1
data.loc[data['Close-Open'] < 0,'C-O'] = -1

data = data[['Date','Time','C-O']]
#크롤링한 데이터를 가져옴 data = 주가데이터 / crawling_data = 크롤링데이터
crawling_data = pd.read_csv("C:/Users/young/210222/휴마시스.csv",encoding="cp949")

list1 = []

#토요일
crawling_data.loc[crawling_data['요일'] == '토', '날짜'] += 2
crawling_data.loc[crawling_data['요일'] == '토', '시간'] = 36
#일요일
crawling_data.loc[crawling_data['요일'] == '일', '날짜'] += 1
crawling_data.loc[crawling_data['요일'] == '일', '시간'] = 36
#주중 장외
crawling_data.loc[crawling_data['시간'] >= 62, '날짜'] += 1
crawling_data.loc[crawling_data['시간'] >= 62, '시간'] = 36
crawling_data.loc[crawling_data['시간'] < 36, '시간'] =36

#상승/하락 구하기.
for i in range(len(crawling_data)):
    try:
        temp = data[data['Time'] == crawling_data['시간'][i]]    
        temp2 = temp[temp['Date'] == crawling_data['날짜'][i]]
        list1.append(temp2['C-O'].values[0])        
    except:        
        list1.append(np.nan)
        
crawling_data['상승/하락'] = list1


#필요한 열만 가져가자
crawling_data = crawling_data[['제목','상승/하락']]

#중복값제거
crawling_data = crawling_data.drop_duplicates(['제목'], keep = 'last') 

#결측치제거
crawling_data.dropna(inplace = True)

#0인값을 없애고 -1을 0으로 바꾼다.
temp1 = crawling_data[crawling_data['상승/하락'] == 0].index
crawling_data = crawling_data.drop(temp1)
crawling_data.loc[crawling_data['상승/하락'] == -1, '상승/하락'] = 0
crawling_data= crawling_data.reset_index(drop=True)
crawling_data.to_csv('C:/Users/young/210222/완료/휴마시스.csv',encoding="",index=False)

In [42]:
removeList2 = ['[주간추천주]', '[현장에서]', '[富를 키우는 투자지표]', '[다시보는 리포트]', '[전문기자칼럼]', '[위클리 마켓]', '[주말n입사지원]',
 '[빅데이터로 본 재테크]', '[주요 크레딧 공지]', '[ 컨콜]', '[잠정실적]', '이데일리', '[Worst]', '[임상의 맥]', '[기업공시]',
 '[이태호의 캐피털마켓 워치]', '[코스피 마감]', '[하나금투 주간추천주]', '[itM-특징주]', '[오늘의 목표주가]',
 '[fnRASSI]', '[이슈타임]', '[주간HOT종목]', '[김대성의 종목분석]', '[뒷북경제]', '[마켓인사이드]', '[한경 팩트체크]', '[뉴스새벽배송]',
 '[이데일리N]', '[김영수의 현장클릭]', '[SK證 주간추천주]', '[코스닥人]', '[이번주 추천주]', "['약'한뉴스]", '[현장클릭]', '[홍길용의 화식열전]',
  '[줌인]', '[어떻게 생각하십니까]', '[동정]', '[미리보는 이데일리 신문]', '[중기info]', '[숫자로 본 K바이오]',
 '[INSIDE]', '[오후시황]', '[인터뷰]', '[fn마켓워치]', '[시그널FOCUS]', '[CB 발행 그후]①', '[fm마켓워치]',
 '[공시]', '[IT선빵!]', '[클릭 e종목]', '[서경스타즈IR]', '[장외로 번진 투자열기]', '[마켓브리핑]', '[2020자본투자대상]',
 '[2020국감]', '[특별기고]', '[머니콕]', '[Q&A]', '[프로필]', '[신년사]', '[특징주]', '[Hot-Line]', '[오늘장 미리보기]', '[포스트IPO]⑩',
  '[일문일답]', '[종목 돋보기]', '[23일 주요 크레딧 공시]', '[itM]', '[주목!e스몰캡]', '[SK증권 주간추천주]', '[줌인 이종목]',
 '[시그널]', '[줌인리더스클럽]', '[속보]', '[마감시황]', '[마켓포인트]', '[마켓뷰]', '[MBN GOLD 시황저격]', '[위클리M&A]', '[마켓인]',
  '[유가증권 메모]', '[단독]', '[31st SRE]', '[강세 토픽]', '[유안타證 주간추천주]', '[다시 보는 리포트]', '[피플]',
 '[마켓인사이트]', '[주목! 딜 종목①]', '[사진]', '[핫이슈]', '[유가증권 기업공시]', '[업종공략]',
 '[조재길의 경제산책]', '[집중분석 이슈타임]', '[종합]', '[주식 초고수는 지금]', '[영상]', '현대상선','KAI']
dataset = pd.read_csv("210219_dataset.csv", encoding="euc-kr")
dataset.dropna(inplace = True)
companydata = pd.read_csv("상장법인목록.csv",encoding="euc-kr")
removeList = companydata['회사명']
temp, temp1 = [], ""
for i in dataset['제목']: 
    temp1 = i
    for j in removeList:
        if j in i:
            temp1 = i.replace(j, '')
            temp1 = temp1.replace(',','')        
    temp.append(temp1)
dataset['제목'] = temp
temp, temp1 = [], ""
for i in dataset['제목']:   
    temp1 = i
    for j in removeList2:
        if j in i:
            temp1 = i.replace(j, '')       
    temp.append(temp1)    
dataset['제목'] = temp

dataset

,제목,상승/하락
0,'일곱 개의 대죄' 글로벌 앱스토어 매출 톱3 등극,1.0
1,“ 기업가치평가 부담 탈피…기존 게임 감소세 관리해야”,0.0
2,고평가 부담 탈피…신작일정별 대응 유효-이베스트,0.0
3,신규 게임 출시·실적 적중률↑…“긍정적”-메리츠,0.0
4,기존 라인업 매출 하향세..목표가↓-KTB,0.0
...,...,...
8193,코로나19에 말라리아 치료제 효과 소식에 강세,0.0
8194,(019170) 현재 +9.63%,0.0
8195,'' 5% 이상 상승 단기·중기 이평선 정배열로 상승세,1.0
8196,'' 5% 이상 상승 전일 외국인 대량 순매도,1.0


In [43]:
dataset.to_csv('dataset1.csv',encoding="",index=False)